# standard

In [31]:
import numpy as np
import matplotlib.pyplot as plt

from pyzzle import Puzzle, FancyPuzzle, Dictionary, ObjectiveFunction, Optimizer, utils, Mask

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
puzzle = FancyPuzzle(Mask.donut_s)
puzzle.import_dict(Dictionary.dataset.pokemon)

In [11]:
obj_func = ObjectiveFunction()
obj_func.register(["circulation", "nwords", "cross_count"])

In [15]:
optimizer = Optimizer()
optimizer.set_method("local_search")

In [16]:
puzzle.compile(obj_func, optimizer)

In [17]:
puzzle.first_solve()

In [18]:
puzzle.show()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,■,■,■,■,■,,,,,,■,■,■,■,■
1,■,■,■,,,,ア,マ,ル,ス,,,■,■,■
2,■,■,,,フ,,ノ,,ギ,,,,ダ,■,■
3,■,,エ,,シ,,プ,,ア,ル,セ,ウ,ス,,■
4,■,,レ,ジ,ギ,ガ,ス,,,,,,ト,,■
5,,,ブ,,バ,,■,■,■,ゴ,ロ,ン,ダ,,ヒ
6,ヌ,オ,ー,,ナ,■,■,■,■,■,ト,,ス,バ,メ
7,,ー,,ビ,,■,■,■,■,■,ム,,,,グ
8,コ,ダ,ッ,ク,,■,■,■,■,■,,ア,メ,タ,マ
9,,イ,,テ,,エ,■,■,■,,,マ,,,


In [19]:
puzzle.solve(epoch=10)

>>> Interim solution


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,■,■,■,■,■,,,,,,■,■,■,■,■
1,■,■,■,,,,ア,マ,ル,ス,,,■,■,■
2,■,■,,,フ,,ノ,,ギ,,,,ダ,■,■
3,■,,エ,,シ,,プ,,ア,ル,セ,ウ,ス,,■
4,■,,レ,ジ,ギ,ガ,ス,,,,,,ト,,■
5,,,ブ,,バ,,■,■,■,ゴ,ロ,ン,ダ,,ヒ
6,ヌ,オ,ー,,ナ,■,■,■,■,■,ト,,ス,バ,メ
7,,ー,,ビ,,■,■,■,■,■,ム,,,,グ
8,コ,ダ,ッ,ク,,■,■,■,■,■,,ア,メ,タ,マ
9,,イ,,テ,,エ,■,■,■,,,マ,,,


>>> Epoch 1/10
    - Replaced(same score): [ 0. 25. 24.] -> [ 0. 25. 24.]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,■,■,■,■,■,,,,,,■,■,■,■,■
1,■,■,■,,,,ア,マ,ル,ス,,,■,■,■
2,■,■,,,フ,,ノ,,ギ,,,,ダ,■,■
3,■,,エ,,シ,,プ,,ア,ル,セ,ウ,ス,,■
4,■,,レ,ジ,ギ,ガ,ス,,,,,,ト,,■
5,,,ブ,,バ,,■,■,■,ゴ,ロ,ン,ダ,,ヒ
6,ヌ,オ,ー,,ナ,■,■,■,■,■,ト,,ス,バ,メ
7,,ー,,ビ,,■,■,■,■,■,ム,,,,グ
8,コ,ダ,ッ,ク,,■,■,■,■,■,,ア,メ,タ,マ
9,,イ,,テ,,エ,■,■,■,,,マ,,,


>>> Epoch 2/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 3/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 4/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 5/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 6/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 7/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 8/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 9/10
    - Stayed: [ 0. 25. 24.]
>>> Epoch 10/10
    - Replaced(same score): [ 0. 25. 24.] -> [ 0. 25. 24.]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,■,■,■,■,■,,,,,,■,■,■,■,■
1,■,■,■,,,,ア,マ,ル,ス,,,■,■,■
2,■,■,,,フ,,ノ,,ギ,,,,ダ,■,■
3,■,,エ,,シ,,プ,,ア,ル,セ,ウ,ス,,■
4,■,,レ,ジ,ギ,ガ,ス,,,,,,ト,,■
5,,,ブ,,バ,,■,■,■,ゴ,ロ,ン,ダ,,ヒ
6,ヌ,オ,ー,,ナ,■,■,■,■,■,ト,,ス,バ,メ
7,,ー,,ビ,,■,■,■,■,■,ム,,,,グ
8,コ,ダ,ッ,ク,,■,■,■,■,■,,ア,メ,タ,マ
9,,イ,,テ,,エ,■,■,■,,,マ,,,


In [20]:
puzzle.show()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,■,■,■,■,■,,,,,,■,■,■,■,■
1,■,■,■,,,,ア,マ,ル,ス,,,■,■,■
2,■,■,,,フ,,ノ,,ギ,,,,ダ,■,■
3,■,,エ,,シ,,プ,,ア,ル,セ,ウ,ス,,■
4,■,,レ,ジ,ギ,ガ,ス,,,,,,ト,,■
5,,,ブ,,バ,,■,■,■,ゴ,ロ,ン,ダ,,ヒ
6,ヌ,オ,ー,,ナ,■,■,■,■,■,ト,,ス,バ,メ
7,,ー,,ビ,,■,■,■,■,■,ム,,,,グ
8,コ,ダ,ッ,ク,,■,■,■,■,■,,ア,メ,タ,マ
9,,イ,,テ,,エ,■,■,■,,,マ,,,


In [81]:
enable_and_cell = np.full(puzzle.cell.shape, " ")
enable_and_cell[puzzle.cover>0] = puzzle.cell[puzzle.cover>0]
enable_and_cell[puzzle.enable==False] = "×"
enable_and_cell[puzzle.mask==False] = "■"
utils.show_2Darray(enable_and_cell)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,■,■,■,■,■,,×,,×,,■,■,■,■,■
1,■,■,■,,×,×,ア,マ,ル,ス,×,,■,■,■
2,■,■,×,,フ,,ノ,,ギ,,,,ダ,■,■
3,■,,エ,,シ,,プ,×,ア,ル,セ,ウ,ス,×,■
4,■,×,レ,ジ,ギ,ガ,ス,×,×,,×,,ト,,■
5,,×,ブ,,バ,,■,■,■,ゴ,ロ,ン,ダ,×,ヒ
6,ヌ,オ,ー,×,ナ,■,■,■,■,■,ト,×,ス,バ,メ
7,,ー,×,ビ,×,■,■,■,■,■,ム,×,×,,グ
8,コ,ダ,ッ,ク,×,■,■,■,■,■,×,ア,メ,タ,マ
9,,イ,,テ,,エ,■,■,■,×,,マ,,,×
